# P2SH MultiSig BIP322
## Role: Creator / Updator (see role definitions [here](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#Roles))

## Workflow diagram

Taken from [BIP174: Partially Signed Bitcoin Transaction Format](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki)

<img src="https://github.com/bitcoin/bips/raw/master/bip-0174/multisig-workflow.svg">


In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Create HD Key from mnemonic

Note: this would typically access some wallet

See [BIP39](https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki) which desribes using mnemonics for derive a HD private key.

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
mnemonic = 'priority orange artist purpose sport blade spike leg calm renew dress figure session play solve fatal spy reform humble carry finger response order print'

In [4]:
hd_rootkey = HDPrivateKey.from_mnemonic(mnemonic)

root_fp = hd_rootkey.fingerprint().hex()

## 2. Select Key to Sign With

See [BIP32](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki) for more information about derivation paths.

In [5]:
bip32_path = "m/45'/0/0/1"

hd_priv_a = hd_rootkey.traverse(bip32_path)
pubkey = hd_priv_a.pub.point


## 3. Fetch Other Signing Parties Public Keys

See [signer_1](./Signer_1.ipynb) and [signer_2](./Signer_2.ipynb) notebooks steps 1 to 3.

Note: the PSBT_creator needs to have a secure communication channel with the signers.

In [6]:
signer_1_pubkey = '02fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b'
signer_2_pubkey = '02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92'


## 4. Create 2-of-3 MultiSig Pay-to-Script Hash

Note: This p2sh_address would likely be something that the signers would want to include in a DID Document.

A p2sh address and associated redeem script is defined in [BIP16](https://github.com/bitcoin/bips/blob/master/bip-0016.mediawiki)

In [7]:
from buidl.script import RedeemScript

# YOU COULD CHANGE THIS
quorum_m=2

redeem_script = RedeemScript.create_p2sh_multisig(
            quorum_m=quorum_m,
            pubkey_hexes=[pubkey.sec().hex(),signer_1_pubkey,signer_2_pubkey],
            sort_keys=False,
        )


script_pubkey = redeem_script.script_pubkey()
p2sh_address = redeem_script.address()

In [8]:
print("The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address : ", p2sh_address)

The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address :  3LnYoUkFrhyYP3V7rq3mhpwALz1XbCY9Uq


## 5. Create the virtual to_spend transaction locked with the p2sh_multisig

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

The message_challenge is the above 2-3 p2sh scriptPubKey

In [9]:
from src.message import create_to_spend_tx

## You could change this
message_to_sign = "This will be a p2sh 2-of-3 multisig BIP 322 signed message"

to_spend_tx = create_to_spend_tx(p2sh_address, message_to_sign)

## 6. Create PSBT to_sign transaction

The Partially Signed Bitcoin Transaction (PSBT) format consists of key-value maps. Each map consists of a sequence of key-value records, terminated by a 0x00 byte

```
 <psbt> := <magic> <global-map> <input-map>* <output-map>*
 <magic> := 0x70 0x73 0x62 0x74 0xFF
 <global-map> := <keypair>* 0x00
 <input-map> := <keypair>* 0x00
 <output-map> := <keypair>* 0x00
 <keypair> := <key> <value>
 <key> := <keylen> <keytype> <keydata>
 <value> := <valuelen> <valuedata>
 ```

Taken from [BIP174 Specification](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#specification), view for more details

### 6.1 Create the to_sign Tx using the existing function for bip322 defined in [message.py](../../edit/src/message.py)

In [11]:
from src.message import create_to_sign_tx

to_sign_tx = create_to_sign_tx(to_spend_tx.hash())

# NOTE: This is a bug in the current create_to_sign_tx function
# It SHOULD be able to know if it is a segwit transaction or not
to_sign_tx.segwit = False

### 6.2 Create PSBT from the to_sign_tx object

In [12]:
from buidl.psbt_helper import create_multisig_psbt
from buidl.psbt import PSBT

fee = 0
# psbt_to_sign_bip322 = create_multisig_psbt(pubkey_records,input_dicts,output_dicts,fee)
psbt_to_sign_bip322 = PSBT.create(to_sign_tx)

### 6.3 Validate PSBT correctly created

In [13]:
psbt_to_sign_bip322.validate()

True

In [16]:
# Notice the PSBT Ins and PSBT_Outs reflect the TX object with some additional (currently empty) fields
print(psbt_to_sign_bip322)


Tx:

tx: 9723b96cd3d5b4af6c93ca302268ff92bf35933fbaee8662a2b465b854ebba99
version: 0
locktime: 0
tx_ins:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
tx_outs:
0:OP_RETURN 


PSBT XPUBS:
{}

Psbt_Ins:
[
TxIn:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
Prev Tx:
None
Prev Output:
None
Sigs:
{}
RedeemScript:
None
WitnessScript:
None
PSBT Pubs:
{}
ScriptSig:
None
Witness:
None
]

Psbt_Outs:
[
TxOut:
0:OP_RETURN 
RedeemScript:
None
WitnessScript
None
PSBT Pubs:
{}
]

Extra:
{}



## 7. Update PSBT with data know by Creator

This includes:
    
- A lookup for the to_spend_tx referenced as an input to the to_sign_tx
- A lookup for creators HD Public key that is part of the p2sh multisig
- A lookup for the redeemscript associated with the p2sh multsig stored in the scriptPubKey of the TxOut for the to_spend_tx


The creator is now acting in the role of the updater (see [updater role in BIP174](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#updater))

### 7.1 Define the tx lookup

This is a map of a transaction has to the actual transaction object

In [17]:
tx_lookup = {}

tx_lookup[to_spend_tx.hash()] = to_spend_tx

### 7.2 Define a pubkey_lookup

This maps from the sec representation of a S256Point to a NamedHDPublicKey object.

Note: a NamedHDPublicKey is a HDPublicKey with additioanl raw path data for the fingerprint of the root HDPublicKey and the derivation path from that root.

In [18]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=hd_priv_a.pub,
    xfp_hex=root_fp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

### 7.3 Define redeem_lookup

This is a map from a hash160 of a redeem script to the actual redeem script

In [19]:
redeem_lookup = {}
redeem_lookup[redeem_script.hash160()] = redeem_script

In [20]:
# This is not set until after the update
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

In [21]:
psbt_to_sign_bip322.update(tx_lookup,pubkey_lookup,redeem_lookup)

In [22]:
# As you can see update populates the scriptPubKey from the to_spend transaction into the tx_obj in the PSBT 
# (the to_sign obj)
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

OP_HASH160 d1763eda828dbe7ae8374d2d7fb6c3381970b272 OP_EQUAL 

## 8. Serialize and Share PSBT with Other Signing Parties

See 

In [23]:
from io import BytesIO

b64_psbt = psbt_to_sign_bip322.serialize_base64()

print("Copy this vvv \n\n")

print(f"b64_psbt = '{b64_psbt}'")

Copy this vvv 


b64_psbt = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAAAEEaVIhAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeIQL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCyEC+uXB9l+Tuy7o40ywy9pfD8ctGNlFEffbtagn43c+3ZJTriIGAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeFHsF6hstAACAAAAAAAAAAAABAAAAAAA='


## 9. Recieve Signed PSBTs from Signer 1 and Signer 2 Notebooks

Note: in this case the 2-3 multisig is satisfied from signatures provided by signer_1 and signer_2. The Creator has no contributed a signature.

You could easily change this notebook so that it is the creator's signature plus one of the signers that satisfy the p2sh multisig instead though.

In [24]:
# Copy this from signer_1.ipynb

signer_1_psbt_b64 = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAACICAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLRzBEAiEAnqF2PrW8sM/5oVrfSpRlCatsF8TSSI45ti+3meo9LGwCH0is4oXB+V89FRxD/l7RESR0UdU/YYZ1SwyAN636w2UBAQRpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuIgYCKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4UewXqGy0AAIAAAAAAAAAAAAEAAAAiBgL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCxQAAAAALQAAgAAAAAAAAAAAAwAAAAAA'

In [25]:
signer_1_psbt = PSBT.parse_base64(signer_1_psbt_b64)

In [26]:
# Copy this from signer_2.ipynb

signer_2_psbt_b64 = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAACICAvrlwfZfk7su6ONMsMvaXw/HLRjZRRH327WoJ+N3Pt2SRzBEAiAfd5b9XIUOhe8Jqx3yrGDCPk0NWjTB5e8zqrIZcsJHlgIgTCNsRXp9LpFY/pCwo5wzC+UavX9KB0ILqlbh4fzv9iABAQRpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuIgYCKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4UewXqGy0AAIAAAAAAAAAAAAEAAAAiBgL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dkhQAHQ2eLQAAgAAAAAAAAAAABgAAAAAA'


In [27]:
signer_2_psbt = PSBT.parse_base64(signer_2_psbt_b64)

## 10. Combine PSBTs 

The creator is now acting in the role of the combiner (see [combiner role in BIP174](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#combiner))

In [28]:
# Before combination the single PSBT_In in the PSBT only has a single sig
len(signer_1_psbt.psbt_ins[0].sigs)

1

In [29]:
signer_1_psbt.combine(signer_2_psbt)

In [30]:
# Afterwards there are two signatures
len(signer_1_psbt.psbt_ins[0].sigs)

2

## 11. Finalize the PSBT to produce the scriptSig on the PSBT Input from the sigs

The creator is now acting in the role of the `input finalizer` (see [input finalizer role in BIP174](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#input-finalizer))

In [33]:
# ScriptSig not populated until after finalization
signer_1_psbt.psbt_ins[0].script_sig

OP_0 30440221009ea1763eb5bcb0cff9a15adf4a946509ab6c17c4d2488e39b62fb799ea3d2c6c021f48ace285c1f95f3d151c43fe5ed111247451d53f6186754b0c8037adfac36501 304402201f7796fd5c850e85ef09ab1df2ac60c23e4d0d5a34c1e5ef33aab21972c2479602204c236c457a7d2e9158fe90b0a39c330be51abd7f4a07420baa56e1e1fceff62001 5221022897c6cb6f2298b72e01e3412c708dbf73513f98e72701433515c260d63fbc9e2102fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b2102fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd9253ae 

In [34]:
signer_1_psbt.finalize()

RuntimeError: Cannot finalize p2sh without a RedeemScript

## 12. Create Final Bitcoin Tx Object from PSBT

The creator is now acting in the role of `transaction extractor` (see [definition](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#transaction-extractor))

In [35]:
# Remember: currently the final_tx() function on the PSBT fails because there are no funds for a tx fee
# BUT for a bip322 we don't need any fee
signer_1_psbt.final_tx()

This transaction won't relay without having a fee of at least 313


RuntimeError: transaction invalid

In [36]:
## Proposed update to the final_tx function in psbt.py

def final_tx(psbt, is_bip322=False):
        """Returns the broadcast-able transaction"""
        # clone the transaction from self.tx_obj
        tx_obj = psbt.tx_obj.clone()
        # determine if the transaction is segwit by looking for a witness field
        #  in any PSBTIn. if so, set tx_obj.segwit = True
        if any([psbt_in.witness for psbt_in in psbt.psbt_ins]):
            tx_obj.segwit = True
        # iterate through the transaction and PSBT inputs together
        #  using zip(tx_obj.tx_ins, self.psbt_ins)
        for tx_in, psbt_in in zip(tx_obj.tx_ins, psbt.psbt_ins):
            # set the ScriptSig of the transaction input
            tx_in.script_sig = psbt_in.script_sig
            # Exercise 7: if the tx is segwit, set the witness as well
            if tx_obj.segwit:
                # witness should be the PSBTIn witness or an empty Witness()
                tx_in.witness = psbt_in.witness or Witness()
        # check to see that the transaction verifies
        if not is_bip322 and not tx_obj.verify():
            raise RuntimeError("transaction invalid")
        # return the now filled in transaction
        return tx_obj

In [37]:
final_to_sign_tx = final_tx(signer_1_psbt, True)

## 13. Check Signature on Input is Valid

In [38]:
final_to_sign_tx.verify_input(0)

True

## 14. Serialize to_sign transaction as BIP322 Signature

The transaction is encoded in FULL format as per the BIP322 specification. As there is no witness for a p2sh signature the SIMPLE format is not possible.

In [39]:
from buidl.helper import base64_encode
bip322_musig = base64_encode(final_to_sign_tx.serialize())

print(bip322_musig)

AAAAAAHNcfHaNfl8f/+ZC2gTr8aF+0KgppYjKM94egaNm/u1ZAAAAAD8AEcwRAIhAJ6hdj61vLDP+aFa30qUZQmrbBfE0kiOObYvt5nqPSxsAh9IrOKFwflfPRUcQ/5e0REkdFHVP2GGdUsMgDet+sNlAUcwRAIgH3eW/VyFDoXvCasd8qxgwj5NDVo0weXvM6qyGXLCR5YCIEwjbEV6fS6RWP6QsKOcMwvlGr1/SgdCC6pW4eH87/YgAUxpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuAAAAAAEAAAAAAAAAAAFqAAAAAA==


## 15. Verify BIP322 Signature against message and address

In [40]:
from src.message import verify_message

In [41]:
verify_message(p2sh_address, bip322_musig, message_to_sign)

True

In [42]:
print("2-3 p2sh :", p2sh_address)

2-3 p2sh : 3LnYoUkFrhyYP3V7rq3mhpwALz1XbCY9Uq


In [43]:
print("message signed : ", message_to_sign)

message signed :  This will be a p2sh 2-of-3 multisig BIP 322 signed message


In [44]:
from buidl.script import address_to_script_pubkey

In [45]:
spubkey = address_to_script_pubkey(p2sh_address)

In [62]:
spubkey

OP_HASH160 d1763eda828dbe7ae8374d2d7fb6c3381970b272 OP_EQUAL 

## Note: from the BIP322 Signature we can retrieve the redeem script which was unlocked by the ScriptSig to produce this BIP322 signature.

I.E. in this case we can learn it was a 2 of 3 multisig with known and identified public keys.

In [52]:
from buidl.helper import base64_decode
from buidl.tx import Tx
import io

decoded_bip322 = base64_decode(bip322_musig)

stream = io.BytesIO(decoded_bip322)
verification_to_sign = Tx.parse(stream)
print(verification_to_sign)


tx: b2c912d6b0964dd185a3d99b1209b6ac50a175f3a8cb299cff994146f00db930
version: 0
locktime: 0
tx_ins:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
tx_outs:
0:OP_RETURN 



In [69]:
from buidl.script import RedeemScript
raw_redeem_script = verification_to_sign.tx_ins[0].script_sig.commands[-1]

RedeemScript.convert(raw_redeem_script)

OP_2 022897c6cb6f2298b72e01e3412c708dbf73513f98e72701433515c260d63fbc9e 02fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b 02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92 OP_3 OP_CHECKMULTISIG 